In [1]:
#import libraries
import os
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from skimage import io
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
class HomerAndBartDatasets(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir, self.images[index])
        image = io.imread(image_path)
        if self.transform is not None:
            image = self.transform(image)
            #image = np.array(image.reshape(1,-1))
            #image = torch.from_numpy(image)
            image = image.view(-1)

        return image

In [3]:
class CombinedDataset(Dataset):
    def __init__(self, dataset_A, dataset_B):
        self.dataset_A = dataset_A
        self.dataset_B = dataset_B

    def __len__(self):
        return len(self.dataset_A) + len(self.dataset_B)

    def __getitem__(self, index):
        data = self.dataset_A[index] if index < len(self.dataset_A) else self.dataset_B[index - len(self.dataset_A)] 
        return data

In [4]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_units_1, hidden_units_2, hidden_units_3, output_size):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_units_1)
        self.layer2 = nn.Linear(hidden_units_1, hidden_units_2)
        self.layer3 = nn.Linear(hidden_units_2, hidden_units_3)
        self.out_layer = nn.Linear(hidden_units_3, output_size)
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        z_1 = self.layer1(x)
        a_1 = self.relu(z_1)
        z_2 = self.layer2(a_1)
        a_2 = self.relu(z_2)
        z_3 = self.layer3(a_2)
        a_3 = self.relu(z_3)
        z_4 = self.out_layer(a_3)
        a_4 = self.sigmoid(z_4)
        return a_4

In [209]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Grayscale(),
    transforms.Resize((64, 64)),
    transforms.Normalize(           
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225])
])

homer_dataset = HomerAndBartDatasets(image_dir = "./homer_bart/Homer", transform=transform)
bart_dataset = HomerAndBartDatasets(image_dir = "./homer_bart/bart", transform=transform)

X = CombinedDataset(homer_dataset, bart_dataset)
y = torch.cat((torch.ones(len(homer_dataset)) ,torch.zeros(len(bart_dataset))),dim=0)

X_train, X_test, y_train, y_test = train_test_split(X ,y, train_size = 0.9, random_state = 10)

input_size = 12288
print(X_train[0])
hidden_units_1 = 128
hidden_units_2 = 512
hidden_units_3 = 128
output_size = 1
model = Model(input_size, hidden_units_1, hidden_units_2, hidden_units_3, output_size)

tensor([2.2489, 2.2489, 2.2489,  ..., 2.6400, 2.6400, 2.6400])


In [226]:
loss_fn = nn.BCELoss()
learning_rate = 0.0001
iterations = 5000
new_X = torch.stack(X_train)
print(new_X.shape)
new_y = y_train.reshape(-1,1)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for iteration in range(iterations):
        y_pred = model(new_X)
        loss = loss_fn(y_pred, new_y)
        optimizer.zero_grad()
        if (iteration)%1000 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

torch.Size([242, 12288])
tensor(5.8249e-12, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(4.5226e-12, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(3.3737e-12, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(2.5013e-12, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(2.1766e-12, grad_fn=<BinaryCrossEntropyBackward0>)


In [227]:
def predict(model, X_test, y_test):
    y_pred = model(X_test)
    #print(y_pred)
    #print(y_test)
    #plt.scatter(y_test.detach().numpy(), y_pred.detach().numpy())
    #plt.show()
    predictions = (y_pred > 0.5).float()
    #print(predictions)
    accuracy = torch.sum((predictions == y_test)).item()/predictions.shape[0]*100
    return accuracy

In [228]:
X_test_new = torch.stack(X_test)
y_test_new = y_test.reshape(-1,1)
acc = predict(model, X_test_new, y_test_new)
print("Accuracy = {:.2f}%".format(acc))

Accuracy = 85.19%
